In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions.normal import Normal

In [7]:
class BayesianLinearRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(BayesianLinearRegression, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        # 重みとバイアスの事前分布のパラメータ
        self.w_mu = nn.Parameter(torch.zeros(input_dim, output_dim))
        self.w_log_sigma = nn.Parameter(torch.zeros(input_dim, output_dim))
        self.b_mu = nn.Parameter(torch.zeros(output_dim))
        self.b_log_sigma = nn.Parameter(torch.zeros(output_dim))
    
    def forward(self, x):
        w_sigma = torch.exp(self.w_log_sigma)
        b_sigma = torch.exp(self.b_log_sigma)
        
        # 重みとバイアスのサンプリング
        w = self.w_mu + w_sigma * torch.randn_like(self.w_mu)
        b = self.b_mu + b_sigma * torch.randn_like(self.b_mu)
        
        return torch.matmul(x, w) + b
    
    def predict_dist(self, x):
        y = self.forward(x)
        
        # 出力の不確実性の計算
        w_sigma = torch.exp(self.w_log_sigma)
        b_sigma = torch.exp(self.b_log_sigma)
        
        # 標準偏差の計算（重みとバイアスの不確実性を考慮）
        output_sigma = torch.sqrt(torch.matmul(x**2, w_sigma**2) + b_sigma**2)
        
        return Normal(y, output_sigma)

class MLPWithBayesianOutput(nn.Module):
    def __init__(self, input_dim):
        super(MLPWithBayesianOutput, self).__init__()
        self.hidden1 = nn.Linear(input_dim, 64)
        self.hidden2 = nn.Linear(64, 64)
        self.hidden3 = nn.Linear(64, 64)
        self.relu = nn.ReLU()
        self.bayesian_output = BayesianLinearRegression(64, 1)
    
    def forward(self, x):
        x = self.relu(self.hidden1(x))
        x = self.relu(self.hidden2(x))
        x = self.relu(self.hidden3(x))
        return self.bayesian_output.predict_dist(x)


In [8]:
X = np.random.rand(100, 5).astype(np.float32)  # 特徴量が5つのデータポイントが100個
y = np.random.rand(100, 1).astype(np.float32)  # 対応するターゲット値

# PyTorchのテンソルに変換
X_tensor = torch.tensor(X)
y_tensor = torch.tensor(y)


# モデルのインスタンスを作成
model = MLPWithBayesianOutput(input_dim=5)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # フォワードパス
    y_dist = model(X_tensor)
    loss = criterion(y_dist.mean, y_tensor)  # 平均に対してMSEを計算
    
    # バックワードパスと最適化
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

model.eval()  # 評価モードに切り替え
with torch.no_grad():
    y_dist = model(X_tensor)
    predicted_mean = y_dist.mean.numpy()
    predicted_std = y_dist.stddev.numpy()

# 予測結果とその不確実性を表示（例）
print(f"Predicted mean: {predicted_mean}")
print(f"Predicted standard deviation: {predicted_std}")

Epoch [100/1000], Loss: 0.1287
Epoch [200/1000], Loss: 0.9459
Epoch [300/1000], Loss: 1.5555
Epoch [400/1000], Loss: 0.1178
Epoch [500/1000], Loss: 0.1040
Epoch [600/1000], Loss: 0.5294
Epoch [700/1000], Loss: 0.1261
Epoch [800/1000], Loss: 0.1386
Epoch [900/1000], Loss: 0.7725
Epoch [1000/1000], Loss: 0.2251
Predicted mean: [[-0.10280591]
 [-0.08365139]
 [-0.10087201]
 [-0.07960093]
 [-0.098579  ]
 [-0.06901768]
 [-0.07021853]
 [-0.09407058]
 [-0.05258003]
 [-0.08405471]
 [-0.07347056]
 [-0.08138773]
 [-0.0616267 ]
 [-0.06410575]
 [-0.0744665 ]
 [-0.0630444 ]
 [-0.08723259]
 [-0.07674316]
 [-0.05297759]
 [-0.06330463]
 [-0.05340222]
 [-0.05488682]
 [-0.08858711]
 [-0.07889247]
 [-0.05075058]
 [-0.07099667]
 [-0.10169153]
 [-0.09803413]
 [-0.06001446]
 [-0.09188728]
 [-0.07692364]
 [-0.06375742]
 [-0.04990736]
 [-0.04822841]
 [-0.10332501]
 [-0.03948778]
 [-0.07067659]
 [-0.10332751]
 [-0.06718957]
 [-0.02621767]
 [-0.06021586]
 [-0.05600971]
 [-0.06708357]
 [-0.09595811]
 [-0.07901987